<a href="https://colab.research.google.com/github/Abhishek0697/Detection-of-Hate-Speech-in-Multimodal-Memes/blob/main/Code/Experiments/BERT%2BResNeXt%2BImage_Captioning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Wed Apr 10 11:52:39 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8              12W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

In [ ]:
class HateMemeDataset(Dataset):
    def __init__(self, labels_file,textfile, img_dir, transform=None):
        """
        Args:
            labels_file (string): Path to the text file with annotations.
            img_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied on an image.
        """
        self.img_labels = []
        with open(labels_file, 'r') as file:
            for line in file:
                filename, label = line.strip().split('\t')
                # Convert label from string to integer (0 for non_hate, 1 for hate)
                label_int = 0 if label == 'non_hate' else 1
                self.img_labels.append((filename, label_int))
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels[idx][0])
        image = Image.open(img_path).convert('RGB')
        label = self.img_labels[idx][1]
        text = self.img_labels[idx][1]
        if self.transform:
            image = self.transform(image)
        return image, text, label


In [ ]:
# Define transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
dataset = HateMemeDataset(labels_file='label.txt',textfile='/content/csv - Sheet1.csv', img_dir='/content/drive/MyDrive/trainlist', transform=transform)


In [ ]:
# Split dataset into train and validation
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])


In [ ]:
# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [ ]:
# Assuming you have the train_dataset and train_loader already defined
dataset_size = len(train_dataset)  # Total number of items in your dataset
batch_size = train_loader.batch_size  # Batch size you're using for the DataLoader

# Calculate the number of batches, rounding up in case of a non-integer result
import math
num_batches = math.ceil(dataset_size / batch_size)

print(f"Dataset size: {dataset_size}")
print(f"Batch size: {batch_size}")
print(f"Number of batches in train_loader: {num_batches}")


Dataset size: 16
Batch size: 1
Number of batches in train_loader: 16


In [ ]:
'''
IMPORTING NECESSARY MODULES
'''

import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset, TensorDataset
import sys
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter


**Device**

In [ ]:
gpu_ids = [7,6]
device = torch.device('cuda:'+ str(gpu_ids[0]) if torch.cuda.is_available() else 'cpu')
print(device)

cuda:7


**Model Definition**

In [ ]:
gpu_ids

[7, 6]

In [ ]:
!pip install torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [ ]:
import torch
import torch.nn as nn

In [ ]:
import torch
from torchvision import transforms
from PIL import Image
from transformers import BertModel, BertTokenizer
import torchvision.models as models

# Load pre-trained BERT model and tokenizer
bert_model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(bert_model_name)
bert_model = BertModel.from_pretrained(bert_model_name)

# Load pre-trained ResNet model
resnet_model = models.resnet50(pretrained=True)

# Define preprocessing transforms for ResNet
preprocess_resnet = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Define preprocessing transforms for BERT
def preprocess_text(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    return inputs

# Function to extract text features from BERT
def extract_text_features(text):
    inputs = preprocess_text(text)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    text_features = outputs.last_hidden_state.mean(dim=1)  # Use mean pooling over tokens
    return text_features

# Function to extract image features from ResNet
def extract_image_features(image_path):
    image = Image.open(image_path)
    input_tensor = preprocess_resnet(image)
    input_batch = input_tensor.unsqueeze(0)  # Add batch dimension
    with torch.no_grad():
        image_features = resnet_model(input_batch)
    image_features = image_features.flatten()  # Flatten the features
    return image_features



In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FusionNet(nn.Module):

    def __init__(self, num_classes, drop_prob=0.1):
        super(FusionNet, self).__init__()

        self.concat = nn.Linear(in_features=768+2048+768, out_features=512)
        self.bn = nn.BatchNorm1d(512)
        self.bn1 = nn.BatchNorm1d(768)
        self.bn2 = nn.BatchNorm1d(2048)
        self.bn3 = nn.BatchNorm1d(768)
        self.dropout = nn.Dropout(drop_prob)
        self.classify = nn.Linear(in_features=512, out_features=num_classes)

    def forward(self, text_features, image_features, caption_features):

        text_features = self.bn1(text_features)
        image_features = self.bn2(image_features)
        caption_features = self.bn3(caption_features)

        # Concatenate the features
        fused_input = torch.cat((text_features, image_features, caption_features), dim=1)

        # Pass through linear layer and apply batch normalization
        x = self.concat(fused_input)
        x = self.bn(x)

        # Apply dropout
        x = self.dropout(x)

        # Apply ReLU activation function
        x = F.relu(x)

        # Classification layer
        x = self.classify(x)

        return x


In [ ]:
print

In [ ]:
print(len(train_loader))
print(train_loader.dataset)

16


In [ ]:
for epoch in range(5):
    # Iterate over training dataset in batches
    for batch in train_loader:
        # Extract image and text data from the batch
        images, texts, labels = batch

        # Preprocess the image and text data
        processed_images = preprocess_images(images)
        processed_texts = preprocess_texts(texts)

        # Forward pass
        predictions = fusion_model(processed_images, processed_texts)

        # Compute the loss
        loss = loss_function(predictions, labels)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

NameError: name 'preprocess_images' is not defined

**Hyperparameters**

In [ ]:
'''
Loss Function
'''
criterion = nn.CrossEntropyLoss()


'''
Optimizer
'''
optimizer = torch.optim.SGD(Fusion_model.parameters(), lr=0.1, weight_decay=1e-4, momentum=0.9)
# optimizer = AdamW(Fusion_model.parameters(), lr = 2e-3, eps = 1e-8)


'''
Number of training epochs.
'''
num_Epochs = 10


# '''
# OneCycleLR
# '''
# max_lr = 0.05
# lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, total_steps=None, epochs=num_Epochs, steps_per_epoch=len(train_dataloader), pct_start=0.3, anneal_strategy='cos', cycle_momentum=True, base_momentum=0.85, max_momentum=0.95, div_factor=25.0, final_div_factor=10000.0, last_epoch=-1)


lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size= 4, gamma = 0.001)

# lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max, eta_min=0, last_epoch=-1)

In [ ]:
model_name = 'ImageCaptioning'
model_path = '.saved_model_checkpoints/'+model_name

In [ ]:
writer = SummaryWriter(model_name)

train(Image_model, Text_model, Fusion_model, train_dataloader, validation_dataloader, criterion, optimizer, lr_scheduler, model_path, writer, device, epochs = num_Epochs)

writer.flush()
writer.close()

In [ ]:
'''
Load saved model from checkpoint
'''
Fusion_model, optimizer, lr_scheduler, train_loss, v_loss, v_acc, epoch = load(model_path, Fusion_model, optimizer, lr_scheduler)


**Evaluate**

In [ ]:
test_classify(Fusion_model, validation_dataloader, criterion, device)

**Predict on Test and generate output.csv**

**Test Dataloader**

In [ ]:
testlist = 'path of test set'

test_dataset = mytestdataset(testlist, name='test')
test_dataloader = data.DataLoader(test_dataset, shuffle= False, batch_size = 32, num_workers=8,pin_memory=True)

In [ ]:
from predict import predict
predict(image_model, text_model, fusion_model, test_dataloader, device)